In [1]:
import xspec

In [23]:
import os
import json
import numpy as np

In [16]:
import astropy.io.fits as pf
import pyregion

In [ ]:
import pyds9
ds = pyds9.DS9('7f000001:53375')

In [17]:
# from fitab.py
def mask_from_region(regfile, refimg):
    """
    Create a pixel mask for regfile based on the image WCS info in refimg.

    Uses the pyregion module. Please keep to using circle, box, and ellipse
    shapes in DS9, fk5 format to avoid unexpected behavior.
    """
    fh = pf.open(refimg)
    reg = pyregion.open(regfile)
    mask = reg.get_mask(hdu=fh[0])
    return mask

# from fitab.py
def check_bgdinfofile(bgdinfofile):
    """
    Check that the background info file has the required items.
    """
    if not os.path.exists(bgdinfofile):
        print('Error: file %s not found.' % bgdinfofile)
        return False

    bgdinfo = json.loads(open(bgdinfofile).read())

    problem = False
    for key in (
        'bgfiles', 'regfiles', 'refimgf', 'bgdapfiles', 'bgddetfiles'
    ):
        if key not in bgdinfo:
            problem = True
            print('%s not found in background info file.' % key)

    # Same number of items in bgfiles and regfiles:
    if len(bgdinfo['bgfiles']) != len(bgdinfo['regfiles']):
        problem = True
        print('bgfiles and regfiles must have the same number of entries.')

    # A and B keys in bgdapfiles and bgddetfiles
    if ('A' not in bgdinfo['bgdapfiles'] or
            'B' not in bgdinfo['bgdapfiles'] or
            'A' not in bgdinfo['bgddetfiles'] or
            'B' not in bgdinfo['bgddetfiles']):
        problem = True
        print('bgdapfiles and bgddetfiles must have A and B keys.')

    # Check files exist
    queue = []
    for _ in bgdinfo['bgfiles']:
        if isinstance(_, str):
            queue.append(_)

    for _ in bgdinfo['regfiles']:
        if isinstance(_, str):
            queue.append(_)

    for _ in bgdinfo['bgddetfiles']['A']:
        if isinstance(_, str):
            queue.append(_)

    for _ in bgdinfo['bgddetfiles']['B']:
        if isinstance(_, str):
            queue.append(_)

    queue.append(bgdinfo['refimgf'])
    queue.append(bgdinfo['bgdapfiles']['A'])
    queue.append(bgdinfo['bgdapfiles']['B'])

    for _ in queue:
        if not os.path.exists(_):
            problem = True
            print('Error: file %s not found.' % _)

    if problem:
        return False
    else:
        return bgdinfo


In [3]:
specdir = '/Users/qw/astro/nustar/IC342_X1/90201039002/event_cl/spec'
bgddir = '/Users/qw/astro/nustar/IC342_X1/90201039002/event_cl/bgd'

In [15]:
ab = 'A'
specfile = 'srcA_sr_g30.pha'
regfile = '/Users/qw/astro/nustar/IC342_X1/90201039002/event_cl/src.reg'

In [26]:
os.chdir(bgddir)
bgdinfo = check_bgdinfofile('bgdinfo.json')
# bgfiles and regfiles must have the same ordering
bgfiles = bgdinfo['bgfiles']
regfiles = bgdinfo['regfiles']
refimgf = bgdinfo['refimgf']
bgdapfiles = bgdinfo['bgdapfiles']
bgddetfiles = bgdinfo['bgddetfiles']

bgdapim = {}
bgdapim['A'] = pf.open(bgdapfiles['A'])[0].data
bgdapim['B'] = pf.open(bgdapfiles['B'])[0].data

bgddetim = {}
bgddetim['A'] = [
    pf.open(bgddetfiles['A'][0])[0].data,
    pf.open(bgddetfiles['A'][1])[0].data,
    pf.open(bgddetfiles['A'][2])[0].data,
    pf.open(bgddetfiles['A'][3])[0].data
]
bgddetim['B'] = [
    pf.open(bgddetfiles['B'][0])[0].data,
    pf.open(bgddetfiles['B'][1])[0].data,
    pf.open(bgddetfiles['B'][2])[0].data,
    pf.open(bgddetfiles['B'][3])[0].data
]

In [27]:
bgdinfo

{'bgfiles': ['bgd1A_sr_g30.pha',
  'bgd1B_sr_g30.pha',
  'bgd2A_sr_g30.pha',
  'bgd2B_sr_g30.pha',
  'bgd3A_sr_g30.pha',
  'bgd3B_sr_g30.pha'],
 'regfiles': ['bgd1A.reg',
  'bgd1B.reg',
  'bgd2A.reg',
  'bgd2B.reg',
  'bgd3A.reg',
  'bgd3B.reg'],
 'refimgf': 'bgdapA.fits',
 'bgdapfiles': {'A': 'bgdapA.fits', 'B': 'bgdapB.fits'},
 'bgddetfiles': {'A': ['det0Aim.fits',
   'det1Aim.fits',
   'det2Aim.fits',
   'det3Aim.fits'],
  'B': ['det0Bim.fits', 'det1Bim.fits', 'det2Bim.fits', 'det3Bim.fits']}}

Make a mask of the spectral region, and find the aperture image weight, and detector area weights.

In [46]:
# Make region mask and calculate ratios
regmask = np.int8(mask_from_region(regfile, bgdinfo['refimgf']))
# Check mask in ds9
ds.set_np2arr(regmask * bgdapim[ab])

1

In [44]:
srcA_bgdapsum = np.sum(regmask * bgdapim[ab])
print(srcA_bgdapsum)

115.4831


In [45]:
srcA_bgddet = [np.sum(regmask * detim) for detim in bgddetim[ab]]
print(srcA_bgddet)

[5386.368350425325, 4.149712517880847, 0.0057823775866781635, 405.93703331414486]


In [ ]:
srcA

In [6]:
os.chdir(specdir)
t = pf.open(specfile)

In [7]:
t.info()

Filename: srcA_sr_g30.pha
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     360   (87, 87)   float32   
  1  SPECTRUM      1 BinTableHDU    110   4096R x 4C   [J, J, I, I]   
  2  GTI           1 BinTableHDU     54   40R x 2C   [1D, 1D]   
  3  REG00101      1 BinTableHDU     81   1R x 6C   [1D, 1D, 16A, 1D, 1D, 1I]   
